# Working with ArcGIS Survey123 reports using the ArcGIS API for Python

<div class="alert alert-block alert-info">This notebook uses the ArcGIS API for Python. For more information, see the <a href="https://developers.arcgis.com/python/">ArcGIS API for Python documentation and guides</a>.</div>

ArcGIS Survey123 reports are an effective way to quickly visualize, interpret, and share survey responses. Reports use a template you can personalize, that's applied to your survey results and shared as either a Microsoft Word or PDF document. 

It's common to deploy a <a href="https://doc.arcgis.com/en/survey123/browser/create-surveys/webhooks.htm" target="_blank">webhook</a> as an automated process to create and share reports with survey results. Although, using a third party webhook provider may not be a feasible workflow due to organizational or security policies. This is where the ArcGIS Survey123 module in the ArcGIS API for Python can be used to acheive a similar goal.

This sample notebook demonstrates ways you can use the ArcGIS Survey123 module to work with reports and covers the following functionality: 

* <a href="#generate-default-template">Generate a default report template</a>
* <a href="#identify-report-templates">Identify report templates associated with a survey</a>
* <a href="#generate-a-single-report">Generate a single report</a>
* <a href="#generate-multiple-reports">Generate multiple reports</a>
* <a href="#generate-multiple-reports-org">Generate multiple reports and save to your organization</a>
* <a href="#list-saved-reports">List saved reports</a>

The first step is to connect to your GIS.

In [1]:
import arcgis
from arcgis.gis import GIS
gis = GIS("home")

Next, a Survey Manager is defined, and a survey is accessed using the form's item ID. A survey in the Survey Manager is a single instance of a survey project that contains the item information and properties and provides access to the underlying survey dataset. For more information on Survey Manager, see the <a href="https://developers.arcgis.com/python/api-reference/arcgis.apps.survey123.html#arcgis.apps.survey123.SurveyManager" target="_blank">API Reference for the ArcGIS API for Python</a>.

In [2]:
survey_manager = arcgis.apps.survey123.SurveyManager(gis)
survey_by_id = survey_manager.get("9f01838a15594cfdbb2eb69fafb60d75")
print(survey_by_id)

<Survey @ Water Quality Inspection>


<a id='identify-report-templates'></a>
### Identify report templates associated with a survey

Using the `report_templates` method any report template already associated with the survey will be returned in a list. For each template in the list the title is printed using the print statement below. 

In [3]:
templates = survey_by_id.report_templates
p = [print(t.title) for t in templates]

Water Quality Inspection_sampleTemplate


<a id='generate-default-template'></a>
### Generating a default report template

In the case where there are not any report templates associated with the survey, a default template can be generated using the `create_report_template` method. The `create_report_template` method creates a simple default template automatically downloads it to the `Temp` folder. The ArcGIS Survey123 module does not support uploading a new report template.

In [4]:
new_template = survey_by_id.create_report_template()
print(new_template)

/tmp/template.docx


<a id='generate-a-single-report'></a>
### Generate a single report

Use the `generate_report` method to generate a single report by specifying a single object ID in the `where` parameter. Since there was only one report template associated with the survey the index of 0 is used. 

The `where` clause follows the same format as the `where` parameter when <a href="https://developers.arcgis.com/rest/services-reference/query-feature-service-layer-.htm" target="_blank">querying a feature service layer</a>.  

In [5]:
report = survey_by_id.generate_report(templates[0], where="objectid=122")
print(report)

/tmp/Water_Quality_Inspection_report_e68c44.docx


<a id='generate-multiple-reports'></a>
### Generate multiple reports

If you are looking to generate reports for multiple object IDs in the survey you can specify `"objectid=1 OR objectid=2"` to generate reports for those two features. Alternatively, omitting the `where` clause will result in reports created for all features in the feature service. In either scenario the output is saved as a ZIP file, and will be stored locally on disk if run through a Jupyter Notebook. 

In [6]:
batch_reports = survey_by_id.generate_report(templates[0])
print(batch_reports)

/tmp/Water_Quality_Inspection_report_7a8ea1.zip


<a id='generate-multiple-reports-org'></a>
### Generate multiple reports and save to your organization as an item

To generate reports and save them in your ArcGIS organization content, add the `report_title` argument. By specifying a `report_title` the reports are saved in your content as a code sample, which is available for download as a ZIP and viewed on a desktop.

<div class="alert alert-block alert-danger">This parameter is required in order to upload your report to your organization. However, the name of your report or code sample will default to <b>survey_name_report_unique_ID</b></a>.</div>

The code sample can then be shared to a group or publicly so that others can download the ZIP and view the reports.

In [7]:
import datetime
nowstring = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
batch_reports = survey_by_id.generate_report(templates[0], report_title="Test_{0}".format(nowstring))
print(batch_reports)

<Item title:"Water_Quality_Inspection_report_928f8a" type:Code Sample owner:NinjaGreen>


<a id='list-saved-reports'></a>
### List saved reports

Finally, use the `reports` property to list the reports stored in the ArcGIS organizational account from the current GIS connection. This shows all reports that are saved in that named user's content.

In [8]:
recentReports = survey_by_id.reports
p = [print(t.title) for t in recentReports]

Manhole_Inspection_report_7844e0
Tree_Health_Survey_statistic.docx
Tree Health Survey - Summary table
Tree_Health_Survey_sampleTemplate_summarytest1.docx
Test_summary1
UC2020 Hydrants Maintenance_OID_summary1
